# Recommender System:

- The last thing to do is to use our saved models to recommend items to users:



### For the requested user:
- Calculate the score for every item.
- Sort the items based on the score and output the top results.


### Check which users exist on the test set

In [25]:
!pip install ipython-autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1837 sha256=da0cc99f4626812482eb2bea7b6cb857cea1f851fd56343242f54655c1c53aad
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
You should consider upgrading via the 'pip install --upgrade pip' command.


In [26]:
#### To measure all running time
# https://github.com/cpcloud/ipython-autotime

%load_ext autotime

In [1]:
import pandas as pd

import pickle
import pandas as pd
import numpy as np

import os

#Keras
from keras.models import load_model
from keras import backend as K

# Tensorflow
import tensorflow as tf

from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


### Set and Check GPUs

In [2]:
def set_check_gpu():
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.per_process_gpu_memory_fraction =1 # allow all of the GPU memory to be allocated
    # for 8 GPUs
    cfg.gpu_options.visible_device_list = "0,1,2,3,4,5,6,7" # "0,1"
    # for 1 GPU
#     cfg.gpu_options.visible_device_list = "0"
    #cfg.gpu_options.allow_growth = True  # # Don't pre-allocate memory; dynamically allocate the memory used on the GPU as-needed
    #cfg.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = K.tf.Session(config=cfg)
    K.set_session(sess)  # set this TensorFlow session as the default session for Keras

    print("* TF version: ", [tf.__version__, tf.test.is_gpu_available()])
    print("* List of GPU(s): ", tf.config.experimental.list_physical_devices() )
    print("* Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU'))) 
  
    
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
    # set for 8 GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7";
    # set for 1 GPU
#     os.environ["CUDA_VISIBLE_DEVICES"] = "0";

    # Tf debugging option
    tf.debugging.set_log_device_placement(True)

    gpus = tf.config.experimental.list_physical_devices('GPU')

    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

#     print(tf.config.list_logical_devices('GPU'))
    print(tf.config.experimental.list_physical_devices('GPU'))
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [3]:
# set_check_gpu()

In [4]:
from sklearn.model_selection import train_test_split

review_data = pd.read_csv('../data/amazon_reviews_us_Shoes_v1_00_help_voted_And_cut_lognTail.csv')
review_data.rename(columns={ 'star_rating': 'score','customer_id': 'user_id', 'user': 'user_name'}, inplace=True)

items = review_data.product_id.unique()
item_map = {i:val for i,val in enumerate(items)}
inverse_item_map = {val:i for i,val in enumerate(items)}
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)
items = review_data.item_id.unique()
print ("We have %d unique items in metadata "%items.shape[0])

users = review_data.user_id.unique()
user_map = {i:val for i,val in enumerate(users)}
inverse_user_map = {val:i for i,val in enumerate(users)}
review_data["old_user_id"] = review_data["user_id"] 
review_data["user_id"] = review_data["user_id"].map(inverse_user_map)

items_reviewed = review_data.product_id.unique()
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)

items_reviewed = review_data.item_id.unique()
users = review_data.user_id.unique()
helpful_votes = review_data.helpful_votes.unique()



ratings_train, ratings_test = train_test_split( review_data, test_size=0.1, random_state=0)

We have 97758 unique items in metadata 


In [5]:
ratings_test.user_id.value_counts().sort_values(ascending=False).head(10)

15136    5
15745    4
2378     4
3374     4
47739    4
5981     4
2943     4
36       4
29962    4
1114     4
Name: user_id, dtype: int64

### ** Create a  recommendation example dataset of 100 users from the test set and all items for each and predict recommendations for them

In [6]:
items = review_data.product_id.unique()
item_map = {i:val for i,val in enumerate(items)}
inverse_item_map = {val:i for i,val in enumerate(items)}
review_data["old_item_id"] = review_data["product_id"] # copying for join with metadata
review_data["item_id"] = review_data["product_id"].map(inverse_item_map)
items = review_data.item_id.unique()
print ("We have %d unique items in metadata "%items.shape[0])

We have 97758 unique items in metadata 


In [7]:
# all_info['description'] = all_info['description'].fillna(all_info['title'].fillna('no_data'))
# all_info['title'] = all_info['title'].fillna(all_info['description'].fillna('no_data').apply(str).str[:20])
# all_info['image'] = all_info['image'].fillna('no_data')
# all_info['price'] = pd.to_numeric(all_info['price'],errors="coerce")
# all_info['price'] = all_info['price'].fillna(all_info['price'].median()) 

In [8]:
review_data.head(n=2)

,user_id,product_id,score,product_title,helpful_votes,old_item_id,item_id,old_user_id
0,0,B00FLH88CO,4,Norwegian Fleece Lined Winter Slippers,9,B00FLH88CO,0,50073594
1,0,B00B3HW97Q,5,2 Pairs Foster Grant Womans Versa Polarized Su...,1,B00B3HW97Q,1,50073594


In [9]:
type(review_data['product_id'].unique())

numpy.ndarray

In [10]:
# creating metadata mappings 
titles = review_data['product_title'].unique()
titles_map = {i:val for i,val in enumerate(titles)}
inverse_titles_map = {val:i for i,val in enumerate(titles)}

# price = review_data['price'].unique()
# price_map = {i:val for i,val in enumerate(price)}
# inverse_price_map = {val:i for i,val in enumerate(price)}

# print ("We have %d prices" %price.shape)
print ("We have %d titles" %titles.shape)


# all_info['price_id'] = all_info['price'].map(inverse_price_map)
review_data['title_id'] = review_data['product_title'].map(inverse_titles_map)

# creating dict from 
item2prices = {}
# for val in review_data[['item_id','price_id']].dropna().drop_duplicates().iterrows():
#     item2prices[val[1]["item_id"]] = val[1]["price_id"]

item2titles = {}
# for val in all_info[['item_id','title_id']].dropna().drop_duplicates().iterrows():
#     item2titles[val[1]["item_id"]] = val[1]["title_id"]
        


# populating the rating dataset with item metadata info
# ratings_train["price_id"] = ratings_train["item_id"].map(lambda x : item2prices[x])
# ratings_train["title_id"] = ratings_train["item_id"].map(lambda x : item2titles[x])


# populating the test dataset with item metadata info
# ratings_test["price_id"] = ratings_test["item_id"].map(lambda x : item2prices[x])
# ratings_test["title_id"] = ratings_test["item_id"].map(lambda x : item2titles[x])


We have 51912 titles


In [11]:
# ratings_test = pd.read_parquet('./data/ratings_test.parquet')
# ratings_train = pd.read_parquet('./data/ratings_train.parquet')

In [12]:
review_data.columns

Index(['user_id', 'product_id', 'score', 'product_title', 'helpful_votes',
       'old_item_id', 'item_id', 'old_user_id', 'title_id'],
      dtype='object')

In [13]:
items = review_data.item_id.unique()
df_items = pd.DataFrame(data=items.flatten(),columns=['item_id'])
df_items = pd.merge(df_items,review_data,how='left',left_on=('item_id'),right_on=('item_id'))

# df_items= df_items.sample(100)

df_items['key'] = 1
print ("We have %d unique items  "%df_items['item_id'].shape[0])
# df_items= df_items[['item_id', 'description', 'category', 'title', 'title_id', 'price', 'price_id', 'brand', 'key']]
df_items= df_items[['item_id', 'user_id', 'product_id', 'score', 'product_title', 'helpful_votes', 'old_item_id', 'old_user_id', 'title_id', 'key']]
print(df_items.shape)
df_items.head(2)

We have 155509 unique items  
(155509, 10)


,item_id,user_id,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,key
0,0,0,B00FLH88CO,4,Norwegian Fleece Lined Winter Slippers,9,B00FLH88CO,50073594,0,1
1,1,0,B00B3HW97Q,5,2 Pairs Foster Grant Womans Versa Polarized Su...,1,B00B3HW97Q,50073594,1,1


### Select 100 users

In [14]:
users = ratings_test.user_id.unique()
df_users = pd.DataFrame(data=users.flatten(),columns=['user_id'])
df_users = pd.merge(df_users,ratings_test,how='left',left_on=('user_id'),right_on=('user_id'))

df_users= df_users.sample(100)

df_users['key'] = 1
print ("We have %d unique users  "%df_users['user_id'].shape[0])
df_users= df_users[['user_id', 'key']]
print(df_users.shape)
df_users.head(2)

We have 100 unique users  
(100, 2)


,user_id,key
10477,46892,1
14172,47272,1


## Merge users and item and items metadata

In [15]:
df_unseenData= pd.merge(df_users, df_items, on='key')
del df_unseenData['key']
print ("We have %d unique records in the recommendation example dataset "%df_unseenData.shape[0])
print(df_unseenData.shape)
df_unseenData.sample(10)

We have 15550900 unique records in the recommendation example dataset 
(15550900, 10)


,user_id_x,item_id,user_id_y,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id
11627960,46500,69829,76245,B00HQE5II2,5,Michael Kors Jet Set Travel Medium Multifuncti...,2,B00HQE5II2,27959005,16965
1547034,54954,91064,90721,B0056RK1EM,5,UGG Women's Belcloud Boot,1,B0056RK1EM,16346400,16674
9922255,1130,73507,47963,B00281NKW4,5,Multi-Color Trendy Striped Hobo/Handbag - Colo...,2,B00281NKW4,43064710,40200
3902333,10311,6176,11194,B00DZBBOJC,2,Vionic Women's Tide Sequins Sandals,1,B00DZBBOJC,24453535,654
5831197,4871,40577,51979,B008K94ZU4,4,Jambu Women's Gem Wedge Pump,3,B008K94ZU4,19236485,22246
7461140,49941,94967,92535,B000O16ROI,5,Cobian Men's Floater Thong Sandal,2,B000O16ROI,35050528,49951
15527420,36989,78695,82450,B00AY65VHM,3,PUMA Men's Sierra Trekker Trail Running Shoe,1,B00AY65VHM,48799498,42359
991914,24730,29260,40845,B00DY8ZBPY,4,Clarks Women's Amelia Avery Wedge Sandal,1,B00DY8ZBPY,33965389,1270
15068230,64889,84425,86342,B00510XY0C,4,Circa Joan & David Women's Qamar Sandal,2,B00510XY0C,48667802,44769
5622251,55779,10544,17822,B00RK40HUW,5,Skechers Performance Men's Go Walk 3 Unwind Sl...,1,B00RK40HUW,819223,8530


In [16]:
df_unseenData.columns

Index(['user_id_x', 'item_id', 'user_id_y', 'product_id', 'score',
       'product_title', 'helpful_votes', 'old_item_id', 'old_user_id',
       'title_id'],
      dtype='object')

In [17]:
from os import listdir
from os.path import isfile, join

mypath = '../models'

onlyfiles = [f.replace('.h5', '') for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

['dense_1_Multiply_50_embeddings_4_epochs_dropout',
 'models.txt',
 'dense_5_Multiply_50_embeddings_10_epochs_dropout',
 'matrix_facto_10_embeddings_100_epochs',
 'dense_5_Meta_Multiply_50_embeddings_10_epochs_dropout',
 'dense_1_Multiply_50_embeddings_100_epochs_dropout']

## Predict the ratings for the items and users in the a recommendation example dataset:

In [18]:
load_path = "../models/"

# models =['dense_1_Multiply_50_embeddings_4_epochs_dropout',
#  'dense_5_Multiply_50_embeddings_10_epochs_dropout',
#  'matrix_facto_10_embeddings_100_epochs',
#  'dense_1_Multiply_50_embeddings_100_epochs_dropout']

# select the best model
models =[
 'dense_5_Multiply_50_embeddings_10_epochs_dropout'
]

# models_meta = [
# 'dense_5_Meta_Multiply_50_embeddings_10_epochs_dropout',
# ]

# for mod in models:
#     model = load_model(load_path+mod+'.h5')
#     df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id'],
#                                                    df_unseenData['item_id'],
#                                                    df_unseenData['price_id'],
#                                                    df_unseenData['title_id']])

for mod in models:
    model = load_model(load_path+mod+'.h5')
    df_unseenData['preds_' + mod] = model.predict([df_unseenData['user_id_x'],
                                                   df_unseenData['item_id']])


df_unseenData.head(2)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.











,user_id_x,item_id,user_id_y,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout
0,46892,0,0,B00FLH88CO,4,Norwegian Fleece Lined Winter Slippers,9,B00FLH88CO,50073594,0,4.043950
1,46892,1,0,B00B3HW97Q,5,2 Pairs Foster Grant Womans Versa Polarized Su...,1,B00B3HW97Q,50073594,1,4.044291


In [20]:
df_unseenData = df_unseenData.drop(columns=['user_id_y'])
df_unseenData = df_unseenData.rename(columns={'user_id_x': 'user_id'})

In [21]:
df_unseenData['user_id'].head(n=2)

0    46892
1    46892
Name: user_id, dtype: int64

In [22]:
df_unseenData.columns

Index(['user_id', 'item_id', 'product_id', 'score', 'product_title',
       'helpful_votes', 'old_item_id', 'old_user_id', 'title_id',
       'preds_dense_5_Multiply_50_embeddings_10_epochs_dropout'],
      dtype='object')

In [23]:
df_unseenData.shape

(15550900, 10)

## Check which users exist on the example set

In [ ]:
# df_unseenData.T

In [24]:
df_unseenData.user_id.value_counts().sort_values(ascending=False).head(5)
df_unseenData[['user_id','preds_dense_5_Multiply_50_embeddings_10_epochs_dropout']].sort_values('preds_dense_5_Multiply_50_embeddings_10_epochs_dropout',ascending=True).head(5)

,user_id,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout
7119915,12638,4.043427
7120839,12638,4.043439
279510,47272,4.043442
7088574,12638,4.043460
7088573,12638,4.043460


## A function that will return recommendation list for a given user

In [27]:
df_unseenData.tail(n=3)

,user_id,item_id,product_id,score,product_title,helpful_votes,old_item_id,old_user_id,title_id,preds_dense_5_Multiply_50_embeddings_10_epochs_dropout
15550897,36989,97756,B000074WV6,5,Nike Men's Shox Limitless,12,B000074WV6,34382909,51910,4.043941
15550898,36989,97757,B000072270,5,ASICS&#174; Men's Hyperdistance,1,B000072270,47730138,51911,4.044184
15550899,36989,97757,B000072270,5,ASICS&#174; Men's Hyperdistance,1,B000072270,47730138,51911,4.044184


time: 12.5 ms


In [139]:
load_path = "../models/"

def get_recommendations(userID , model_scr, df_Data):
    if userID not in df_Data['user_id'].values:
        print("\nUser ID not found %d" %userID)
        return userID
       
#     print("\nRecommendations for user id %d Name: %s is:" % (userID, df_Data.loc[df_Data['user_id'] == userID, 'user_name'].values[0]))
   
    df_output=df_Data.loc[df_Data['user_id'] == userID][['item_id','product_title','helpful_votes', model_scr,    
     ]].sort_values(model_scr,ascending=False).set_index('item_id')
#     print(df_output)
    
    df_output.rename(columns={model_scr: 'score'}, inplace=True)
    df_output = df_output.sort_values(by=['score'], ascending=False)
    
    #add ASIN form item_id
#     df_output['product_id'] = df_Data['item_id'].apply(item_map)
    
    return df_output.head(n=20)

time: 2.72 ms


### Recommend items to a given user

- Using dense_5_Multiply_50_embeddings_10_epochs_dropout

In [140]:
df_unseenData.columns

Index(['user_id', 'item_id', 'product_id', 'score', 'product_title',
       'helpful_votes', 'old_item_id', 'old_user_id', 'title_id',
       'preds_dense_5_Multiply_50_embeddings_10_epochs_dropout'],
      dtype='object')

time: 3.2 ms


In [141]:
#######  User ID: 36989                       
df_output = get_recommendations(userID=36989,
                                model_scr='preds_dense_5_Multiply_50_embeddings_10_epochs_dropout', 
                                df_Data=df_unseenData)
df_output.head(10)
# df_output

,product_title,helpful_votes,score
item_id,,,
3776,KEEN Women's Voyageur Hiking Shoe,1,4.050804
3776,KEEN Women's Voyageur Hiking Shoe,2,4.050804
3776,KEEN Women's Voyageur Hiking Shoe,1,4.050804
5850,swedish hasbeens Women's Fideli Platform Sandal,1,4.048142
5850,swedish hasbeens Women's Fideli Platform Sandal,1,4.048142
7223,Naturalizer Women's Jailene Slingback Sandal,3,4.048050
7223,Naturalizer Women's Jailene Slingback Sandal,4,4.048050
7957,Anuschka 382 Hobo,1,4.047860
7957,Anuschka 382 Hobo,1,4.047860


time: 109 ms


## Make predictions for another user using another model:

In [142]:
# #######  User ID
# user_id = 12638
# df_output = get_recommendations(userID=user_id,
#                                 model_scr='preds_dense_1_Multiply_50_embeddings_4_epochs_dropout',
#                                 df_Data=df_unseenData)
# df_output

time: 755 µs


In [143]:
df_output.columns

Index(['product_title', 'helpful_votes', 'score'], dtype='object')

time: 2.6 ms


In [144]:
df_output = df_output.reset_index()
df_output['user_id'] = user_id
df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'product_title']]

time: 8.41 ms


In [145]:
df_output.head()

,user_id,item_id,score,asin,url,product_title
0,71152,3776,4.050804,B00138Z6MK,https://www.amazon.com/dp/B00138Z6MK,KEEN Women's Voyageur Hiking Shoe
1,71152,3776,4.050804,B00138Z6MK,https://www.amazon.com/dp/B00138Z6MK,KEEN Women's Voyageur Hiking Shoe
2,71152,3776,4.050804,B00138Z6MK,https://www.amazon.com/dp/B00138Z6MK,KEEN Women's Voyageur Hiking Shoe
3,71152,5850,4.048142,B00JMYSSBI,https://www.amazon.com/dp/B00JMYSSBI,swedish hasbeens Women's Fideli Platform Sandal
4,71152,5850,4.048142,B00JMYSSBI,https://www.amazon.com/dp/B00JMYSSBI,swedish hasbeens Women's Fideli Platform Sandal


time: 9.92 ms


In [146]:
unseenUser_list = df_unseenData['user_id'].values
print(len(unseenUser_list))

15550900
time: 1.17 ms


In [147]:
unique_review = list(set(unseenUser_list.tolist()))

time: 1.16 s


In [148]:
len(unique_review)

100

time: 22.6 ms


### select 100 users from unseen data

In [149]:
all_predicted_df = pd.DataFrame()

for user_id in unique_review:
    print("selected 100 user_id:", user_id)
    df_output = get_recommendations(userID=user_id ,model_scr='preds_dense_5_Multiply_50_embeddings_10_epochs_dropout',df_Data=df_unseenData)
    df_output = df_output.reset_index()
    df_output['user_id'] = user_id
    df_output['asin'] = df_output['item_id'].apply(lambda x : item_map[x])
    df_output['url'] = df_output['item_id'].apply(lambda x : 'https://www.amazon.com/dp/'+item_map[x])
    df_output = df_output[['user_id','item_id', 'score', 'asin', 'url', 'product_title']]
    
    #concat
    all_predicted_df = all_predicted_df.append(df_output)

selected 100 user_id: 17412
selected 100 user_id: 29200
selected 100 user_id: 20498
selected 100 user_id: 52243
selected 100 user_id: 34329
selected 100 user_id: 68130
selected 100 user_id: 13349
selected 100 user_id: 11302
selected 100 user_id: 82982
selected 100 user_id: 77370
selected 100 user_id: 46652
selected 100 user_id: 7740
selected 100 user_id: 33852
selected 100 user_id: 576
selected 100 user_id: 10311
selected 100 user_id: 82504
selected 100 user_id: 38986
selected 100 user_id: 6742
selected 100 user_id: 61018
selected 100 user_id: 44125
selected 100 user_id: 67165
selected 100 user_id: 9828
selected 100 user_id: 74341
selected 100 user_id: 1130
selected 100 user_id: 67182
selected 100 user_id: 626
selected 100 user_id: 92791
selected 100 user_id: 75895
selected 100 user_id: 36989
selected 100 user_id: 23166
selected 100 user_id: 30342
selected 100 user_id: 31879
selected 100 user_id: 59026
selected 100 user_id: 24730
selected 100 user_id: 92831
selected 100 user_id: 47272


In [150]:
all_predicted_df.shape

(2000, 6)

time: 3.33 ms


In [151]:
all_predicted_df.columns

Index(['user_id', 'item_id', 'score', 'asin', 'url', 'product_title'], dtype='object')

time: 3.99 ms


In [152]:
all_predicted_df = all_predicted_df.drop_duplicates() 
all_predicted_df = all_predicted_df.reset_index()
all_predicted_df.drop(columns=['index'])

,user_id,item_id,score,asin,url,product_title
0,17412,3776,4.045459,B00138Z6MK,https://www.amazon.com/dp/B00138Z6MK,KEEN Women's Voyageur Hiking Shoe
1,17412,7223,4.045054,B005N4OKNG,https://www.amazon.com/dp/B005N4OKNG,Naturalizer Women's Jailene Slingback Sandal
2,17412,20054,4.045036,B008Y1YZVW,https://www.amazon.com/dp/B008Y1YZVW,Qupid Women's Relax-70 Two Way Fold Down Sexy ...
3,17412,5850,4.045024,B00JMYSSBI,https://www.amazon.com/dp/B00JMYSSBI,swedish hasbeens Women's Fideli Platform Sandal
4,17412,16906,4.045009,B004CJ4WG8,https://www.amazon.com/dp/B004CJ4WG8,Tifosi Wisp T Dual-Lens Sunglasses
5,17412,3888,4.044853,B00ZZSJKCQ,https://www.amazon.com/dp/B00ZZSJKCQ,Vionic Womens Action Heritage Slip On Walking ...
6,17412,7957,4.044788,B00N3531X2,https://www.amazon.com/dp/B00N3531X2,Anuschka 382 Hobo
7,17412,52796,4.044777,B007H5SEBI,https://www.amazon.com/dp/B007H5SEBI,Sebago Women's Claremont Brogue Oxford
8,17412,17700,4.044762,B00EU7R7V4,https://www.amazon.com/dp/B00EU7R7V4,FitFlop Women's Frou Dress Sandal
9,29200,3776,4.049302,B00138Z6MK,https://www.amazon.com/dp/B00138Z6MK,KEEN Women's Voyageur Hiking Shoe


time: 47.1 ms


In [153]:
all_predicted_df.shape

(909, 7)

time: 3.68 ms


In [154]:
# Shoes_for_100_users_per_100_products_prediction_Ver2
# all_predicted_df.to_csv('Shoes_for_100_users_per_100_products_prediction_Ver2.csv', header=True, index=False)

#Shoes_for_100_users_per_50_products_prediction_Ver2.csv
# all_predicted_df.to_csv('Shoes_for_100_users_per_50_products_prediction_Ver2.csv', header=True, index=False)

#Shoes_for_100_users_per_20_products_prediction_Ver2.csv
all_predicted_df.to_csv('Shoes_for_100_users_per_20_products_prediction_Ver2.csv', header=True, index=False)

time: 18.4 ms
